In [ ]:
# 출처 : https://huggingface.co/microsoft/Phi-3-vision-128k-instruct

import time
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

start_time = time.time()
model_id = "microsoft/Phi-3-vision-128k-instruct" 

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation='flash_attention_2') # use _attn_implementation='eager' to disable flash attention

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 

# 소요된 시간을 계산합니다.
end_time = time.time()
formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'*time:{formatted_elapsed_time}')

In [ ]:
# 로컬 폴더에 저장 (예: ./model/Phi-3-vision-128k-instruct 폴더에 저장)
folder = "./model/Phi-3-vision-128k-instruct"
model.save_pretrained(folder, safe_serialization=False)
processor.save_pretrained(folder)

In [2]:
import time
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

start_time = time.time()

# 로컬 파일에서 모델과 프로세서 불러오기
folder = "./model/Phi-3-vision-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(folder, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation='flash_attention_2')
processor = AutoProcessor.from_pretrained(folder, trust_remote_code=True)

# 소요된 시간을 계산합니다.
end_time = time.time()
formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'*time:{formatted_elapsed_time}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/models/auto/image_processing_auto.py:507: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


*time:7.70


In [3]:
import time
from PIL import Image 
import requests 

start_time = time.time()
    
messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    #{"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."}, 
    #{"role": "user", "content": "Provide insightful questions to spark discussion."} 
    {"role": "user", "content": "Describe the image content in detail."} 
] 

#url = "https://assets-c4akfrf5b4d3f4b7.z01.azurefd.net/assets/2024/04/BMDataViz_661fb89f3845e.png" 
#image = Image.open(requests.get(url, stream=True).raw) 
image = Image.open("./data/car.jpg")

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.5, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

 # 소요된 시간을 계산합니다.
end_time = time.time()
formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'*time:{formatted_elapsed_time}')
    
print(response)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


*time:4.20
The image features a vintage Volkswagen Beetle car with a teal and white color scheme, parked on a cobblestone street. The car is positioned in front of a yellow building with two doors, one of which is brown and the other is white. The sky is clear, suggesting it might be a sunny day.


In [4]:
#!pip install googletrans==4.0.0-rc1
from googletrans import Translator
def translate_google(text:str, source_lang:str, target_lang:str):
    translator = Translator()
    
    start_time = time.time()
    res = translator.translate(text, src=source_lang, dest=target_lang)

    # 소요된 시간을 계산합니다.
    end_time = time.time()
    formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
    print(f'time:{formatted_elapsed_time}')

    print(res)
    print()
    response = res.text.strip('"')
    return response

text = translate_google(text=response, source_lang='en', target_lang='ko')
print(text)

time:0.83
Translated(src=en, dest=ko, text=이 이미지는 청록색과 흰색 구성표가있는 빈티지 폭스 바겐 딱정벌레 차량을 특징으로하며 조약돌 거리에 주차되었습니다.차는 두 개의 문이있는 노란 건물 앞에 위치하고 있으며 그 중 하나는 갈색이고 다른 하나는 흰색입니다.하늘은 깨끗하고 화창한 날 일 수 있음을 시사합니다., pronunciation=i imijineun cheonglogsaeggwa huinsaeg guseongpyogaissneun bintiji pogseu bagen ttagjeongbeolle chalyang-eul teugjing-eulohamyeo joyagdol geolie juchadoeeossseubnida. chaneun du gaeui mun-iissneun nolan geonmul ap-e wichihago iss-eumyeo geu jung hananeun galsaeg-igo daleun hananeun huinsaeg-ibnida. haneul-eun kkaekkeushago hwachanghan nal il su iss-eum-eul sisahabnida., extra_data="{'confiden...")

이 이미지는 청록색과 흰색 구성표가있는 빈티지 폭스 바겐 딱정벌레 차량을 특징으로하며 조약돌 거리에 주차되었습니다.차는 두 개의 문이있는 노란 건물 앞에 위치하고 있으며 그 중 하나는 갈색이고 다른 하나는 흰색입니다.하늘은 깨끗하고 화창한 날 일 수 있음을 시사합니다.
